In [2]:
# from google.colab.patches import cv2_imshow
import numpy as np
import cv2 as cv
import os

In [ ]:
!unzip "/content/drive/MyDrive/ISIA Lab/Dataset/CLICK_unprocess.zip"

# Farneback

In [ ]:
path = fr"notebooks\robot"
pathSave = fr"notebooks\robot_cv_flow"
numberImages = len(os.listdir(fr"{path}\left"))

In [ ]:
i = 1; error=0
frame = cv.imread(fr"{path}\left\{i}.jpg")
prvsL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

frame = cv.imread(fr"{path}\right\{i}.jpg")
prvsR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame)
hsv[...,1] = 255
i+=1

while i <= numberImages:
    #Left
    frame = cv.imread(fr"{path}\left\{i}.jpg")
    nextL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    flow = cv.calcOpticalFlowFarneback(prvsL,nextL, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    rgb_flowL = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)

    rgb_flowL = cv.normalize(rgb_flowL, dst=None, alpha=0, beta=255,norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

    #Right
    frame = cv.imread(fr"{path}\right\{i}.jpg")
    nextR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    flow = cv.calcOpticalFlowFarneback(prvsR,nextR, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    rgb_flowR = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)

    rgb_flowR = cv.normalize(rgb_flowR, dst=None, alpha=0, beta=255,norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

    if(np.any(rgb_flowL != 0) and np.any(rgb_flowR != 0)):
        cv.imwrite(f"{pathSave}/left/{i-1}.jpg", rgb_flowL)
        cv.imwrite(f"{pathSave}/right/{i-1}.jpg", rgb_flowR)
        prvsL = nextL
        prvsR = nextR
        i+=1
    else:
        error+=1
        if(error==20):
            print(f"error: {i}")
            error=0
            i+=1
            frame = cv.imread(fr"{path}\left\{i}.jpg")
            prvsL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            frame = cv.imread(fr"{path}\right\{i}.jpg")
            prvsR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            i+=1

# Flow net

In [ ]:
!pip3 install --quiet ml4a

     |████████████████████████████████| 1.1 MB 30.8 MB/s 
     |████████████████████████████████| 561 kB 63.7 MB/s 
     |████████████████████████████████| 108 kB 45.6 MB/s 
     |████████████████████████████████| 411.5 MB 8.0 kB/s 
     |████████████████████████████████| 235 kB 29.7 MB/s 
     |████████████████████████████████| 132 kB 79.2 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
     |████████████████████████████████| 829 kB 68.8 MB/s 
     |████████████████████████████████| 140 kB 70.7 MB/s 
     |████████████████████████████████| 50 kB 8.3 MB/s 
     |████████████████████████████████| 503 kB 68.2 MB/s 
     |████████████████████████████████| 722 kB 57.8 MB/s 
     |████████████████████████████████| 3.8 MB 68.2 MB/s 
     |████████████████████████████████| 100.1 MB 27 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8

In [ ]:
import os
import shutil

from ml4a.models import flownet
from ml4a import image

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3743744/45929032 bytes (8.2%)7921664/45929032 bytes (17.2%)12140544/45929032 bytes (26.4%)16252928/45929032 bytes (35.4%)20054016/45929032 bytes (43.7%)24207360/45929032 bytes (52.7%)28418048/45929032 bytes (61.9%)32653312/45929032 bytes (71.1%)36823040/45929032 bytes (80.2%)41033728/45929032 bytes (89.3%)45047808/45929032 bytes (98.1%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [ ]:
pathData = r"/content/CLICK"
pathSave = r"/content/process"

In [ ]:
!rm -rf /content/process

In [ ]:
id = 0
directories = os.listdir(pathData)
for directory in directories:
    print(directory)
    images = os.listdir(f"{pathData}/{directory}/data/left")
    img0l = image.load_image(f"{pathData}/{directory}/data/left/0.jpg")
    img0r = image.load_image(f"{pathData}/{directory}/data/right/0.jpg")
    index = 1
    while index < len(images):
      # Left
      img1l = image.load_image(f"{pathData}/{directory}/data/left/{index}.jpg")
      rgb_flowl = flownet.run(img0l, img1l, to_rgb=True)

      # Right
      img1r = image.load_image(f"{pathData}/{directory}/data/right/{index}.jpg")
      rgb_flowr = flownet.run(img0r, img1r, to_rgb=True) 

      # Save
      image.save(rgb_flowl, f"{pathSave}/left/{id}.jpg")
      image.save(rgb_flowr, f"{pathSave}/right/{id}.jpg")

      img0l = img1l
      img0r = img1r
      index+=1
      id+=1

Kitchen


Downloading...
From: https://drive.google.com/uc?id=1jbWiY1C_nqAUJRYZu7mwzV6CK7ugsa5v
To: /root/.ml4a/FlowNetPytorch/flownets_EPE1.951.pth.tar
100%|██████████| 155M/155M [00:03<00:00, 49.9MB/s]


Corridor2
SmallMeetingsRoom
Corridor1
Room1
SeminarRoom
BigRoom
MeetingsRoom
DevRoom


In [ ]:
shutil.make_archive('CLICK_flow', 'zip', '/content/process')

'/content/CLICK_flow.zip'